In [ ]:
import gradio as gr
import torch
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
from PIL import Image
import io
from src.utils.misc import Rescale
from src.utils.utils_io import png_loader


In [ ]:
# Load your PyTorch model

if torch.cuda.is_available():
    model = torch.load("./best-model.ckpt")
    # model = torch.load("./src/ui/best-model.ckpt")
else:
    model = torch.load("./best-model.ckpt", map_location=torch.device('cpu'))
    # model = torch.load("./src/ui/best-model.ckpt", map_location=torch.device('cpu'))
# model = torch.load("../model/BEST-epoch=23-step=864006.ckpt")
# model.eval()

In [ ]:
# Function to convert PNG to WebP
def prepare_sample(left_image, right_imate):

    left_image = png_loader(left_image)
    right_imate = png_loader(right_imate)

    sample = {
        'left': left_image, 'right': right_imate, 
        }
    transforms = [Rescale()]

    torch_sample = ToTensor()(sample)
    for transform in transforms:
        torch_sample = transform(torch_sample)

    return torch_sample

def disparity_estimation(left_image, right_image):

    sample = prepare_sample(left_image, right_image)
    # disparity_map = model(sample['left'], sample['right'])
    with torch.no_grad():
        disparity_map = model(sample)
    return disparity_map

In [ ]:
# Create a Gradio interface
iface = gr.Interface(
    fn=disparity_estimation,
    inputs=[
        gr.inputs.Image(type="pil", label="Left Image (PNG)"),
        gr.inputs.Image(type="pil", label="Right Image (PNG)"),
    ],
    outputs=gr.outputs.Image(type="pil", label="Disparity Map (PNG)"),
    title="Disparity Estimation",
    description="Upload two PNG images for left and right views to estimate disparity.",
)

In [ ]:
# Start the Gradio UI
iface.launch()

In [ ]:
iface.close()